# Advanced Remote tensor execution tools

In [2]:
import torch as th
from torch import optim, nn

In [3]:
# toy dataset
data = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [4]:
#Simple linear model
model = nn.Linear(2, 1)

In [5]:
#create optimizer

opt = optim.SGD(params=model.parameters(), lr = 0.01)

In [6]:
opt.zero_grad()

In [7]:
pred = model(data)

In [8]:
loss = ((pred - target) ** 2).sum()

In [9]:
loss.backward()

In [10]:
opt.step()

In [11]:
loss.data

tensor(3.3706)

In [12]:
#create method to learn the model

def train(iteration = 20):
    for i in range(iteration):
        opt.zero_grad()
        pred = model(data)
        loss = ((pred - target) ** 2).sum()
        loss.backward()
        opt.step()
        print("In %i loss:"%i, "%f" %loss.data)
        

In [13]:
train()

In 0 loss: 2.902265
In 1 loss: 2.534071
In 2 loss: 2.242548
In 3 loss: 2.009802
In 4 loss: 1.822193
In 5 loss: 1.669321
In 6 loss: 1.543257
In 7 loss: 1.437948
In 8 loss: 1.348774
In 9 loss: 1.272206
In 10 loss: 1.205544
In 11 loss: 1.146722
In 12 loss: 1.094156
In 13 loss: 1.046629
In 14 loss: 1.003204
In 15 loss: 0.963159
In 16 loss: 0.925934
In 17 loss: 0.891093
In 18 loss: 0.858298
In 19 loss: 0.827281


In [14]:
# Use two virtual workers and send half datasets to each
import syft as sy

hook = sy.TorchHook(th)
viper = sy.VirtualWorker(hook, id = "viper")
quassi = sy.VirtualWorker(hook, id = "quassi")

In [15]:
data_viper = data[:2].send(viper)
target_viper = target[:2].send(viper)

data_quassi = data[2:].send(quassi)
target_quassi = target[2:].send(quassi)

In [16]:
datasets = [(data_viper, target_viper), (data_quassi, target_quassi)]

In [17]:
model = nn.Linear(2, 1)

In [18]:
opt = optim.SGD(params = model.parameters(), lr = 0.01)

In [19]:
_data, _target = datasets[0]
model = model.send(_data.location)

In [20]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [21]:
train()

tensor(1.5514, requires_grad=True)
tensor(0.0497, requires_grad=True)
tensor(0.0698, requires_grad=True)
tensor(0.0426, requires_grad=True)
tensor(0.0337, requires_grad=True)
tensor(0.0265, requires_grad=True)
tensor(0.0210, requires_grad=True)
tensor(0.0164, requires_grad=True)
tensor(0.0134, requires_grad=True)
tensor(0.0103, requires_grad=True)
tensor(0.0087, requires_grad=True)
tensor(0.0067, requires_grad=True)
tensor(0.0057, requires_grad=True)
tensor(0.0044, requires_grad=True)
tensor(0.0038, requires_grad=True)
tensor(0.0030, requires_grad=True)
tensor(0.0026, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0015, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0010, requires_grad=True)
tensor(0.0009, requires_grad=True)
tensor(0.0008, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0004, requires_grad=True)
tensor(0.0003, requi

In [46]:
viper.clear_objects()

<VirtualWorker id:viper #objects:0>

In [47]:
quassi.clear_objects()

<VirtualWorker id:quassi #objects:0>

In [48]:
viper._objects

{}

In [49]:
x = th.tensor([1, 2, 3, 4]).send(viper)  # x is pointer here

In [50]:
x = x.send(quassi) # x is pointer addressing to pointer

In [51]:
quassi._objects

{19071640047: (Wrapper)>[PointerTensor | quassi:19071640047 -> viper:31595827496]}

In [52]:
x #now x points to quassi

(Wrapper)>[PointerTensor | me:65761226050 -> quassi:19071640047]

In [53]:
y = x + x

In [54]:
y #still pointing to quassi, as it first executes to it

(Wrapper)>[PointerTensor | me:72110475577 -> quassi:88660062268]

In [55]:
viper._objects #contains 2 pointers

{31595827496: tensor([1, 2, 3, 4]), 74437627574: tensor([2, 4, 6, 8])}

In [56]:
quassi._objects

{19071640047: (Wrapper)>[PointerTensor | quassi:19071640047 -> viper:31595827496],
 88660062268: (Wrapper)>[PointerTensor | quassi:88660062268 -> viper:74437627574]}

In [57]:
viper.clear_objects()
quassi.clear_objects()

<VirtualWorker id:quassi #objects:0>

In [58]:
#send pointer of x to viper and its pointer to quassi
x = th.tensor([1, 2, 3, 4]).send(viper).send(quassi)

In [60]:
quassi._objects #see the pointers pointing to 

{62053936875: (Wrapper)>[PointerTensor | quassi:62053936875 -> viper:10501984931]}

In [61]:
x = x.get(quassi)

In [62]:
x

(Wrapper)>[PointerTensor | me:62053936875 -> viper:10501984931]

In [63]:
# now see objects of quassi
quassi._objects

{}

In [64]:
viper._objects

{10501984931: tensor([1, 2, 3, 4])}

> ## This is because of automatic garbage collection feature of pysyft. But there is still viper has data. We need to delete the original pointer to automatically delete all pointers pointing towards actual data.

In [73]:
viper.clear_objects()
quassi.clear_objects()

#send pointer of x to viper and its pointer to quassi
x = th.tensor([1, 2, 3, 4]).send(viper).send(quassi)



In [74]:
viper._objects

{59845145184: tensor([1, 2, 3, 4])}

In [75]:
quassi._objects

{97118002576: (Wrapper)>[PointerTensor | quassi:97118002576 -> viper:59845145184]}

In [76]:
del x

In [78]:
viper._objects

{}

In [79]:
quassi._objects

{}

> < Now automatically collected all garbages.